### 다대일 방식의 감정 분석

In [1]:
import numpy as np
import os
import shutil
import tensorflow as tf
from sklearn.metrics import accuracy_score, confusion_matrix

In [4]:
def download_and_read(url):
    local_file = url.split('/')[-1]
    local_file = local_file.replace("%20", " ")
    p = tf.keras.utils.get_file(local_file, url, extract = True, cache_dir = ".")
    local_folder = os.path.join("datasets", local_file.split(".")[0])
    labeled_sentences = []
    for labeled_filename in os.listdir(local_folder):
        if labeled_filename.endswith("_labelled.txt"):
            with open(os.path.join(local_folder, labeled_filename), "r") as f:
                for line in f:
                    sentence, label = line.strip().split('\t')
                    labeled_sentences.append((sentence, label))
    return labeled_sentences

In [5]:
labeled_sentences = download_and_read("https://archive.ics.uci.edu/ml/machine-learning-databases/" + 
                                     "00331/sentiment%20labelled%20sentences.zip")
sentences = [s for (s, l) in labeled_sentences]
labels = [int(l) for (s, l) in labeled_sentences]

90112/84188 [================================] - 0s 3us/step


In [6]:
# 말뭉치의 단어를 정수로 매핑한 것을 어휘라고 한다. 따라서 문장을 토큰화하고 어휘를 만들어야 한다.
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(sentences)
vocab_size = len(tokenizer.word_counts)
print("vocabulary size: {:d}".format(vocab_size))
word2idx = tokenizer.word_index
idx2word = {v:k for (k, v) in word2idx.items()}

vocabulary size: 5271


In [8]:
seq_lengths = np.array([len(s.split()) for s in sentences])
print([(p, np.percentile(seq_lengths, p)) for p in [75, 80, 90, 95, 99, 100]])

[(75, 16.0), (80, 18.0), (90, 22.0), (95, 26.0), (99, 36.0), (100, 71.0)]


In [9]:
max_seqlen = 64

sentences_as_ints = tokenizer.texts_to_sequences(sentences)
sentences_as_ints = tf.keras.preprocessing.sequence.pad_sequences(sentences_as_ints, maxlen = max_seqlen)
labels_as_ints = np.array(labels)
dataset = tf.data.Dataset.from_tensor_slices((sentences_as_ints, labels_as_ints))

In [11]:
dataset = dataset.shuffle(10000)
test_size = len(sentences)//3

val_size = (len(sentences) - test_size) // 10
test_dataset = dataset.take(test_size)
val_dataset = dataset.skip(test_size).take(val_size)
train_dataset = dataset.skip(test_size + val_size)

batch_size = 64
train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

In [13]:
class SentimentAnalysisModel(tf.keras.Model):
    def __init__(self, vocab_size, max_seqlen, **kwargs):
        super(SentimentAnalysisModel, self).__init__(**kwargs)
        self.embedding = tf.keras.layers.Embedding(vocab_size, max_seqlen)
        self.bilstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(max_seqlen))
        self.dense = tf.keras.layers.Dense(64, activation = "relu")
        self.out = tf.keras.layers.Dense(1, activation = "sigmoid")
        
    def call(self, x):
        x = self.embedding(x)
        x = self.bilstm(x)
        x = self.dense(x)
        out = self.out(x)
        return out
    
model = SentimentAnalysisModel(vocab_size+1, max_seqlen)
model.build(input_shape = (batch_size, max_seqlen))
model.summary()

Model: "sentiment_analysis_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  337408    
_________________________________________________________________
bidirectional_1 (Bidirection multiple                  66048     
_________________________________________________________________
dense_2 (Dense)              multiple                  8256      
_________________________________________________________________
dense_3 (Dense)              multiple                  65        
Total params: 411,777
Trainable params: 411,777
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [18]:
logs_dir = os.path.join("./logs")
best_model_file = os.path.join("./best_model.h5")
checkpoint = tf.keras.callbacks.ModelCheckpoint(best_model_file, save_weights_only = True,
                                               save_best_only = True)
tensorboard = tf.keras.callbacks.TensorBoard(log_dir = logs_dir)
num_epochs = 10
history = model.fit(train_dataset, epochs = num_epochs, validation_data = val_dataset,
                   callbacks = [checkpoint, tensorboard])

Train for 29 steps, validate for 4 steps
Epoch 1/10
29/29 [==============================] - 3s 107ms/step - loss: 0.2037 - accuracy: 0.9206 - val_loss: 0.1581 - val_accuracy: 0.9400
Epoch 2/10
29/29 [==============================] - 3s 103ms/step - loss: 0.1508 - accuracy: 0.9467 - val_loss: 0.0836 - val_accuracy: 0.9800
Epoch 3/10
29/29 [==============================] - 3s 103ms/step - loss: 0.1096 - accuracy: 0.9650 - val_loss: 0.1037 - val_accuracy: 0.9700
Epoch 4/10
29/29 [==============================] - 3s 103ms/step - loss: 0.0607 - accuracy: 0.9811 - val_loss: 0.0555 - val_accuracy: 0.9800
Epoch 5/10
29/29 [==============================] - 3s 106ms/step - loss: 0.0714 - accuracy: 0.9794 - val_loss: 0.0156 - val_accuracy: 1.0000
Epoch 6/10
29/29 [==============================] - 3s 103ms/step - loss: 0.0393 - accuracy: 0.9906 - val_loss: 0.0400 - val_accuracy: 0.9850
Epoch 7/10
29/29 [==============================] - 3s 101ms/step - loss: 0.0294 - accuracy: 0.9967 - val_l